In [1]:
# import packages
import sys 
sys.path.append('scripts/')
import json
import random
import commands as cmd
import utils
from memory import get_memory, get_supported_memory_backends
import chat
from colorama import Fore, Style
from spinner import Spinner
import time
import speak
from config import Config
from json_parser import fix_and_parse_json
from ai_config import AIConfig
import traceback
import yaml
import argparse
from logger import logger
import logging
from prompt import get_prompt

/home/codespace/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
cfg = Config()

In [3]:
cfg.set_debug_mode(True)

In [4]:
cfg.set_smart_llm_model(cfg.fast_llm_model)

In [5]:
get_supported_memory_backends()

['local', 'redis', 'pinecone']

In [6]:
cfg.memory_backend = 'local'

In [7]:
logger.set_level(logging.DEBUG if cfg.debug_mode else logging.INFO)

In [8]:
ai_name = ""

In [9]:
# make first prompt
def construct_prompt():
    """Construct the prompt for the AI to respond to"""
    config = AIConfig.load()
    if config.ai_name:
        logger.typewriter_log(
            f"Welcome back! ",
            Fore.GREEN,
            f"Would you like me to return to being {config.ai_name}?",
            speak_text=True)
        should_continue = utils.clean_input(f"""Continue with the last settings?
Name:  {config.ai_name}
Role:  {config.ai_role}
Goals: {config.ai_goals}
Continue (y/n): """)
        if should_continue.lower() == "n":
            config = AIConfig()

    if not config.ai_name:
        config = prompt_user()
        config.save()

    # Get rid of this global:
    global ai_name
    ai_name = config.ai_name

    full_prompt = config.construct_full_prompt()
    return full_prompt
prompt = construct_prompt()
print(prompt)

Welcome  back!  Would you like me to return to being writer_2?
You are writer_2, an AI design to write articles
Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.

GOALS:

1. Search 10 news items about AutoGPT on the Internet
2. Summarize the news into a 1000-word "news.txt".
3. None
4. None
5. None


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Google Search: "google", args: "input": "<search>"
2. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"
3. Start GPT Agent: "sta

In [10]:
full_message_history = []
result = None
next_action_count = 0
# Make a constant:
user_input = "Determine which next command to use, and respond using the format specified above:"
# Initialize memory and make sure it is empty.
# this is particularly important for indexing and referencing pinecone memory

In [11]:
# memory init
memory = get_memory(cfg, init=True)
memory

In [12]:
print('Using memory of type: ' + memory.__class__.__name__)

Using memory of type: LocalCache


In [13]:
loop_count = 0

In [14]:
assistant_reply = chat.chat_with_ai(
    prompt,
    user_input,
    full_message_history,
    memory,
    cfg.fast_token_limit) 

print(assistant_reply)

  Token limit: 4000
  Memory Stats: (0, (0, 1536))
  Token limit: 4000
  Send Token Count: 898
  Tokens remaining for response: 3102
  ------------ CONTEXT SENT TO AI ---------------
  System: The current time and date is Wed Apr 19 01:14:08 2023
  
  System: This reminds you of these events from your past:



  
  User: Determine which next command to use, and respond using the format specified above:
  
  ----------- END OF CONTEXT ----------------
{
    "thoughts": {
        "text": "I will begin by searching the internet for recent news about AutoGPT.",
        "reasoning": "I need up-to-date information to write my article on AutoGPT.",
        "plan": "- Search the internet using Google search for recent news about Autogpt\n- Read through them, and select the top 10 relevant news articles\n- Summarize the news articles into a 1000-word news.txt file.",
        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done late

In [16]:
def print_assistant_thoughts(assistant_reply):
    """Prints the assistant's thoughts to the console"""
    global ai_name
    global cfg
    try:
        try:
            # Parse and print Assistant response
            assistant_reply_json = fix_and_parse_json(assistant_reply)
        except json.JSONDecodeError as e:
            logger.error("Error: Invalid JSON in assistant thoughts\n", assistant_reply)
            assistant_reply_json = attempt_to_fix_json_by_finding_outermost_brackets(assistant_reply)
            assistant_reply_json = fix_and_parse_json(assistant_reply_json)

        # Check if assistant_reply_json is a string and attempt to parse it into a JSON object
        if isinstance(assistant_reply_json, str):
            try:
                assistant_reply_json = json.loads(assistant_reply_json)
            except json.JSONDecodeError as e:
                logger.error("Error: Invalid JSON\n", assistant_reply)
                assistant_reply_json = attempt_to_fix_json_by_finding_outermost_brackets(assistant_reply_json)

        assistant_thoughts_reasoning = None
        assistant_thoughts_plan = None
        assistant_thoughts_speak = None
        assistant_thoughts_criticism = None
        assistant_thoughts = assistant_reply_json.get("thoughts", {})
        assistant_thoughts_text = assistant_thoughts.get("text")

        if assistant_thoughts:
            assistant_thoughts_reasoning = assistant_thoughts.get("reasoning")
            assistant_thoughts_plan = assistant_thoughts.get("plan")
            assistant_thoughts_criticism = assistant_thoughts.get("criticism")
            assistant_thoughts_speak = assistant_thoughts.get("speak")

        logger.typewriter_log(f"{ai_name.upper()} THOUGHTS:", Fore.YELLOW, assistant_thoughts_text)
        logger.typewriter_log("REASONING:", Fore.YELLOW, assistant_thoughts_reasoning)

        if assistant_thoughts_plan:
            logger.typewriter_log("PLAN:", Fore.YELLOW, "")
            # If it's a list, join it into a string
            if isinstance(assistant_thoughts_plan, list):
                assistant_thoughts_plan = "\n".join(assistant_thoughts_plan)
            elif isinstance(assistant_thoughts_plan, dict):
                assistant_thoughts_plan = str(assistant_thoughts_plan)

            # Split the input_string using the newline character and dashes
            lines = assistant_thoughts_plan.split('\n')
            for line in lines:
                line = line.lstrip("- ")
                logger.typewriter_log("- ", Fore.GREEN, line.strip())

        logger.typewriter_log("CRITICISM:", Fore.YELLOW, assistant_thoughts_criticism)
        # Speak the assistant's thoughts
        if cfg.speak_mode and assistant_thoughts_speak:
            speak.say_text(assistant_thoughts_speak)

        return assistant_reply_json
    except json.decoder.JSONDecodeError as e:
        logger.error("Error: Invalid JSON\n", assistant_reply)
        if cfg.speak_mode:
            speak.say_text("I have received an invalid JSON response from the OpenAI API. I cannot ignore this response.")

    # All other errors, return "Error: + error message"
    except Exception as e:
        call_stack = traceback.format_exc()
        logger.error("Error: \n", call_stack)
print_assistant_thoughts(assistant_reply)

WRITER_2 THOUGHTS:  I will begin by searching the internet for recent news about AutoGPT.
REASONING:  I need up-to-date information to write my article on AutoGPT.
PLAN: 
-  Search the internet using Google search for recent news about Autogpt
-  Read through them, and select the top 10 relevant news articles
-  Summarize the news articles into a 1000-word news.txt file.
CRITICISM:  I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.


{'thoughts': {'text': 'I will begin by searching the internet for recent news about AutoGPT.',
  'reasoning': 'I need up-to-date information to write my article on AutoGPT.',
  'plan': '- Search the internet using Google search for recent news about Autogpt\n- Read through them, and select the top 10 relevant news articles\n- Summarize the news articles into a 1000-word news.txt file.',
  'criticism': 'I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.',
  'speak': 'I will perform a Google search to find recent news about AutoGPT.'},
 'command': {'name': 'google', 'args': {'input': 'recent news about AutoGPT'}}}

In [17]:
# parse assistant reply to get command name and arguments
# command_name, arguments = cmd.get_command(
                # attempt_to_fix_json_by_finding_outermost_brackets(assistant_reply))

def attempt_to_fix_json_by_finding_outermost_brackets(json_string):
    if cfg.speak_mode and cfg.debug_mode:
      speak.say_text("I have received an invalid JSON response from the OpenAI API. Trying to fix it now.")
    logger.typewriter_log("Attempting to fix JSON by finding outermost brackets\n")

    try:
        # Use regex to search for JSON objects
        import regex
        json_pattern = regex.compile(r"\{(?:[^{}]|(?R))*\}")
        json_match = json_pattern.search(json_string)

        if json_match:
            # Extract the valid JSON object from the string
            json_string = json_match.group(0)
            logger.typewriter_log(title="Apparently json was fixed.", title_color=Fore.GREEN)
            if cfg.speak_mode and cfg.debug_mode:
               speak.say_text("Apparently json was fixed.")
        else:
            raise ValueError("No valid JSON object found")

    except (json.JSONDecodeError, ValueError) as e:
        if cfg.speak_mode:
            speak.say_text("Didn't work. I will have to ignore this response then.")
        logger.error("Error: Invalid JSON, setting it to empty JSON now.\n")
        json_string = {}

    return json_string

attempt_to_fix_json_by_finding_outermost_brackets(assistant_reply)

Attempting to fix JSON by finding outermost brackets 
Apparently json was fixed. 


'{\n    "thoughts": {\n        "text": "I will begin by searching the internet for recent news about AutoGPT.",\n        "reasoning": "I need up-to-date information to write my article on AutoGPT.",\n        "plan": "- Search the internet using Google search for recent news about Autogpt\\n- Read through them, and select the top 10 relevant news articles\\n- Summarize the news articles into a 1000-word news.txt file.",\n        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",\n        "speak": "I will perform a Google search to find recent news about AutoGPT."\n    },\n    "command": {\n        "name": "google",\n        "args": {\n            "input": "recent news about AutoGPT"\n        }\n    }\n}'

In [18]:
assistant_reply

'{\n    "thoughts": {\n        "text": "I will begin by searching the internet for recent news about AutoGPT.",\n        "reasoning": "I need up-to-date information to write my article on AutoGPT.",\n        "plan": "- Search the internet using Google search for recent news about Autogpt\\n- Read through them, and select the top 10 relevant news articles\\n- Summarize the news articles into a 1000-word news.txt file.",\n        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",\n        "speak": "I will perform a Google search to find recent news about AutoGPT."\n    },\n    "command": {\n        "name": "google",\n        "args": {\n            "input": "recent news about AutoGPT"\n        }\n    }\n}'

In [19]:
print(assistant_reply)

{
    "thoughts": {
        "text": "I will begin by searching the internet for recent news about AutoGPT.",
        "reasoning": "I need up-to-date information to write my article on AutoGPT.",
        "plan": "- Search the internet using Google search for recent news about Autogpt\n- Read through them, and select the top 10 relevant news articles\n- Summarize the news articles into a 1000-word news.txt file.",
        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",
        "speak": "I will perform a Google search to find recent news about AutoGPT."
    },
    "command": {
        "name": "google",
        "args": {
            "input": "recent news about AutoGPT"
        }
    }
}


In [20]:
command_name, arguments = cmd.get_command(
                attempt_to_fix_json_by_finding_outermost_brackets(assistant_reply))

command_name, arguments

Attempting to fix JSON by finding outermost brackets 
Apparently json was fixed. 


('google', {'input': 'recent news about AutoGPT'})

In [21]:
# run command
rlt = cmd.execute_command(command_name, arguments)

In [22]:
rlt

['https://www.marktechpost.com/2023/04/16/breaking-down-autogpt-what-it-is-its-features-limitations-artificial-general-intelligence-agi-and-impact-of-autonomous-agents-on-generative-ai/',
 'https://interestingengineering.com/innovation/meet-autogpt-internet-is-crazy-about',
 'https://levelup.gitconnected.com/autogpt-the-new-kid-on-the-ai-block-thats-changing-everything-6d6ee8cf169f',
 'https://www.euronews.com/next/2023/04/14/make-some-room-chatgpt-rival-ai-chatbot-autogpt-has-landed-this-is-what-we-know-about-it-s',
 'https://fortune.com/2023/04/15/babyagi-autogpt-openai-gpt-4-autonomous-assistant-agi/',
 'https://pub.towardsai.net/the-ai-revolution-how-auto-gpt-unleashes-a-new-era-of-automation-and-creativity-2008aa2ca6ae',
 'https://www.tomsguide.com/news/auto-gpt',
 'https://autogpt.net/amazing-use-cases-for-auto-gpt-on-twitter/']

In [23]:
result = f"Command {command_name} returned: {rlt}"
result

"Command google returned: ['https://www.marktechpost.com/2023/04/16/breaking-down-autogpt-what-it-is-its-features-limitations-artificial-general-intelligence-agi-and-impact-of-autonomous-agents-on-generative-ai/', 'https://interestingengineering.com/innovation/meet-autogpt-internet-is-crazy-about', 'https://levelup.gitconnected.com/autogpt-the-new-kid-on-the-ai-block-thats-changing-everything-6d6ee8cf169f', 'https://www.euronews.com/next/2023/04/14/make-some-room-chatgpt-rival-ai-chatbot-autogpt-has-landed-this-is-what-we-know-about-it-s', 'https://fortune.com/2023/04/15/babyagi-autogpt-openai-gpt-4-autonomous-assistant-agi/', 'https://pub.towardsai.net/the-ai-revolution-how-auto-gpt-unleashes-a-new-era-of-automation-and-creativity-2008aa2ca6ae', 'https://www.tomsguide.com/news/auto-gpt', 'https://autogpt.net/amazing-use-cases-for-auto-gpt-on-twitter/']"

In [24]:
user_input == "GENERATE NEXT COMMAND JSON"
memory_to_add = f"Assistant Reply: {assistant_reply} " \
                f"\nResult: {result} " \
                f"\nHuman Feedback: {user_input} "
memory_to_add

'Assistant Reply: {\n    "thoughts": {\n        "text": "I will begin by searching the internet for recent news about AutoGPT.",\n        "reasoning": "I need up-to-date information to write my article on AutoGPT.",\n        "plan": "- Search the internet using Google search for recent news about Autogpt\\n- Read through them, and select the top 10 relevant news articles\\n- Summarize the news articles into a 1000-word news.txt file.",\n        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",\n        "speak": "I will perform a Google search to find recent news about AutoGPT."\n    },\n    "command": {\n        "name": "google",\n        "args": {\n            "input": "recent news about AutoGPT"\n        }\n    }\n} \nResult: Command google returned: [\'https://www.marktechpost.com/2023/04/16/breaking-down-autogpt-what-it-is-its-features-limitations-artificial-general-intelligence-agi-and-impact-of-autonomous

In [25]:
user_input == "GENERATE NEXT COMMAND JSON"
memory_to_add = f"Assistant Reply: {assistant_reply} " \
                f"\nResult: {result} " \
                f"\nHuman Feedback: {user_input} "
print(memory_to_add)

Assistant Reply: {
    "thoughts": {
        "text": "I will begin by searching the internet for recent news about AutoGPT.",
        "reasoning": "I need up-to-date information to write my article on AutoGPT.",
        "plan": "- Search the internet using Google search for recent news about Autogpt\n- Read through them, and select the top 10 relevant news articles\n- Summarize the news articles into a 1000-word news.txt file.",
        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",
        "speak": "I will perform a Google search to find recent news about AutoGPT."
    },
    "command": {
        "name": "google",
        "args": {
            "input": "recent news about AutoGPT"
        }
    }
} 
Result: Command google returned: ['https://www.marktechpost.com/2023/04/16/breaking-down-autogpt-what-it-is-its-features-limitations-artificial-general-intelligence-agi-and-impact-of-autonomous-agents-on-generati

In [26]:
memory.add(memory_to_add)
memory

In [27]:
# ?add log out put file
full_message_history.append(chat.create_chat_message("system", result))
logger.typewriter_log("SYSTEM: ", Fore.YELLOW, result)

SYSTEM:   Command google returned: ['https://www.marktechpost.com/2023/04/16/breaking-down-autogpt-what-it-is-its-features-limitations-artificial-general-intelligence-agi-and-impact-of-autonomous-agents-on-generative-ai/', 'https://interestingengineering.com/innovation/meet-autogpt-internet-is-crazy-about', 'https://levelup.gitconnected.com/autogpt-the-new-kid-on-the-ai-block-thats-changing-everything-6d6ee8cf169f', 'https://www.euronews.com/next/2023/04/14/make-some-room-chatgpt-rival-ai-chatbot-autogpt-has-landed-this-is-what-we-know-about-it-s', 'https://fortune.com/2023/04/15/babyagi-autogpt-openai-gpt-4-autonomous-assistant-agi/', 'https://pub.towardsai.net/the-ai-revolution-how-auto-gpt-unleashes-a-new-era-of-automation-and-creativity-2008aa2ca6ae', 'https://www.tomsguide.com/news/auto-gpt', 'https://autogpt.net/amazing-use-cases-for-auto-gpt-on-twitter/']


In [28]:
full_message_history

[{'role': 'user',
  'content': 'Determine which next command to use, and respond using the format specified above:'},
 {'role': 'assistant',
  'content': '{\n    "thoughts": {\n        "text": "I will begin by searching the internet for recent news about AutoGPT.",\n        "reasoning": "I need up-to-date information to write my article on AutoGPT.",\n        "plan": "- Search the internet using Google search for recent news about Autogpt\\n- Read through them, and select the top 10 relevant news articles\\n- Summarize the news articles into a 1000-word news.txt file.",\n        "criticism": "I should be mindful not to spend too much time on this task as I have other tasks that need to be done later.",\n        "speak": "I will perform a Google search to find recent news about AutoGPT."\n    },\n    "command": {\n        "name": "google",\n        "args": {\n            "input": "recent news about AutoGPT"\n        }\n    }\n}'},
 {'role': 'system',
  'content': "Command google returne

In [29]:
full_message_history[0]

{'role': 'user',
 'content': 'Determine which next command to use, and respond using the format specified above:'}